# Poseidon Permutation Truncation Polynomial Model Demonstration

In [1]:
using Oscar
include("Poseidon.jl")

generate_compression_polynomials (generic function with 1 method)

## Poseidon Instance

In [2]:
q = 10007 # d = 3
#q = 10009 # d = 5
field = GF(q)
n = 2
r_f = 2
r_p = 2

poseidon = Poseidon_constructor(field=field, n=n, r_f=r_f, r_p=r_p);

Poseidon parameters
Field: Prime field of characteristic 10007
d: 3
n: 2
r_f: 2
r_p: 2
Matrix:
[5816 2038; 3736 911]
Constants:
[4106 7333 7868 2386 7980 5612; 6776 1791 6542 9602 4363 6703]


In [3]:
x = matrix(map(i -> rand(field), 1:n))

[1355]
[1954]

In [4]:
compr_val = compression(poseidon, x)

1-element Vector{FqFieldElem}:
 6924

In [5]:
polynomials = generate_compression_polynomials(poseidon,
                                               compr_val=compr_val);

sep = repeat("-", 70)
println(sep)

for i in 1:(2 * r_f + r_p)
    for j in 1:n
        println(polynomials[n * (i - 1) + j])
        println(sep)
    end
end

Compression value: FqFieldElem[6924]
substitution: true
----------------------------------------------------------------------
5816*x_5_2^3 + 2038*x_out_1^3 + 10006*x_in_1 + 4106
----------------------------------------------------------------------
3736*x_5_2^3 + 911*x_out_1^3 + 10006*x_in_2 + 6776
----------------------------------------------------------------------
5816*x_in_1^3 + 2038*x_in_2^3 + 10006*x_1_1 + 7333
----------------------------------------------------------------------
3736*x_in_1^3 + 911*x_in_2^3 + 10006*x_1_2 + 1791
----------------------------------------------------------------------
5816*x_1_1^3 + 2038*x_1_2 + 10006*x_2_1 + 7868
----------------------------------------------------------------------
3736*x_1_1^3 + 911*x_1_2 + 10006*x_2_2 + 6542
----------------------------------------------------------------------
5816*x_2_1^3 + 2038*x_2_2 + 10006*x_3_1 + 2386
----------------------------------------------------------------------
3736*x_2_1^3 + 911*x_2_2 + 10006

## Guess In Last Round

In [6]:
variables = gens(parent(polynomials[1]));
N = length(variables)
guess = map(var -> var - rand(field), variables[N - 2 * n + 1:N - Int64(3 / 2 * n)])

1-element Vector{FqMPolyRingElem}:
 x_4_2 + 4425

In [7]:
gb = groebner_basis(ideal(polynomials))

for poly in gb
    println(poly)
    println(sep)
end

x_2_2 + 4022*x_3_1 + 7710*x_3_2 + 587
----------------------------------------------------------------------
x_1_2 + 4022*x_2_1 + 2073*x_3_1 + 7487*x_3_2 + 899
----------------------------------------------------------------------
x_out_1^3 + 4022*x_in_1 + 7710*x_in_2 + 805
----------------------------------------------------------------------
x_5_2^3 + 9576*x_out_1^3 + 2483*x_in_2 + 6966
----------------------------------------------------------------------
x_4_2^3 + 7710*x_5_1 + 5505*x_5_2 + 9327*x_out_1 + 9200
----------------------------------------------------------------------
x_4_1^3 + 9576*x_4_2^3 + 2483*x_5_1 + 9647*x_5_2 + 539*x_out_1 + 8099
----------------------------------------------------------------------
x_3_2^3 + 4022*x_4_1 + 7710*x_4_2 + 1693
----------------------------------------------------------------------
x_3_1^3 + 9576*x_3_2^3 + 2483*x_4_2 + 4252
----------------------------------------------------------------------
x_2_1^3 + 9576*x_2_2 + 2483*x_3_2 + 4915
--

In [8]:
gb = groebner_basis(ideal(polynomials) + ideal(guess))

for poly in gb
    println(poly)
    println(sep)
end

x_5_1 + 760*x_5_2 + 7450*x_out_1 + 5802
----------------------------------------------------------------------
x_4_2 + 4425
----------------------------------------------------------------------
x_2_2 + 4022*x_3_1 + 7710*x_3_2 + 587
----------------------------------------------------------------------
x_1_2 + 4022*x_2_1 + 2073*x_3_1 + 7487*x_3_2 + 899
----------------------------------------------------------------------
x_out_1^3 + 4022*x_in_1 + 7710*x_in_2 + 805
----------------------------------------------------------------------
x_5_2^3 + 9576*x_out_1^3 + 2483*x_in_2 + 6966
----------------------------------------------------------------------
x_4_1^3 + 2483*x_5_1 + 9647*x_5_2 + 539*x_out_1 + 3134
----------------------------------------------------------------------
x_3_2^3 + 4022*x_4_1 + 8813
----------------------------------------------------------------------
x_3_1^3 + 9576*x_3_2^3 + 4663
----------------------------------------------------------------------
x_2_1^3 + 9576*x

Compute the vector space dimension.

In [9]:
A, _ = quo(parent(gb[1]), ideal(gb))
D = factor(vector_space_dimension(A))

1 * 3^9

## Guess In Partial Rounds

In [10]:
guess = map(i -> variables[n * (r_f + r_p - i - 1) + 1] - rand(field), 1:Int64(n / 2))

1-element Vector{FqMPolyRingElem}:
 x_2_1 + 8477

In [11]:
gb = groebner_basis_f4(ideal(polynomials) + ideal(guess), nr_thrds=16, info_level=2);


Legend for f4 information
--------------------------------------------------------
deg       current degree of pairs selected in this round
sel       number of pairs selected in this round
pairs     total number of pairs in pair list
mat       matrix dimensions (# rows x # columns)
density   density of the matrix
new data  # new elements for basis in this round
          # zero reductions during linear algebra
time(rd)  time of the current f4 round in seconds given
          for real and cpu time
--------------------------------------------------------

deg     sel   pairs        mat          density            new data         time(rd) in sec (real|cpu)
------------------------------------------------------------------------------------------------------
  3       7       7      15 x 25         15.20%        7 new       0 zero         0.03 | 0.36         
  3       1       1       8 x 12         27.08%        1 new       0 zero         0.01 | 0.16         
  3       1       1       3


--------------- INPUT DATA ---------------
#variables                      13
#equations                      13
#invalid equations               0
field characteristic         10007
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                        16
info level                       2
generate pbm files               0
------------------------------------------

---------------- TIMINGS ----------------
overall(elapsed)        0.29 sec
overall(cpu)            3.39 sec
select                  0.00 sec   0.0%
symbolic prep.          0.00 sec   0.0%
update                  0.00 sec   0.0%
convert                 0.00 sec   0.0%
linear algebra          0.00 sec   0.0%
reduce gb               0.00 sec   0.0%
--------------------------------

In [12]:
for poly in gb
    println(poly)
    println(sep)
end

x_3_1 + 4117*x_3_2 + 5944
----------------------------------------------------------------------
x_2_2 + 714*x_3_2 + 542
----------------------------------------------------------------------
x_2_1 + 8477
----------------------------------------------------------------------
x_1_2 + 8917*x_3_2 + 6966
----------------------------------------------------------------------
x_4_1*x_4_2 + 8478*x_4_2^2 + 8253*x_3_2*x_5_1 + 7411*x_4_1*x_5_1 + 7401*x_4_2*x_5_1 + 3590*x_3_2*x_5_2 + 8293*x_4_1*x_5_2 + 493*x_4_2*x_5_2 + 8217*x_3_2*x_out_1 + 6155*x_4_1*x_out_1 + 8885*x_4_2*x_out_1 + 8372*x_3_2 + 9241*x_4_1 + 5504*x_4_2 + 5895*x_5_1 + 5999*x_5_2 + 6697*x_out_1 + 3296
----------------------------------------------------------------------
x_3_2*x_4_2 + 5247*x_4_2^2 + 3843*x_3_2*x_5_1 + 7787*x_4_1*x_5_1 + 6065*x_4_2*x_5_1 + 6667*x_3_2*x_5_2 + 8319*x_4_1*x_5_2 + 3053*x_4_2*x_5_2 + 822*x_3_2*x_out_1 + 1395*x_4_1*x_out_1 + 7795*x_4_2*x_out_1 + 1913*x_3_2 + 1805*x_4_1 + 5632*x_4_2 + 6418*x_5_1 + 1156*x_5_

Compute the vector space dimension.

In [13]:
A, _ = quo(parent(gb[1]), ideal(gb))
D = factor(vector_space_dimension(A))

1 * 3^7

## Guess Second To Last Round

In [14]:
guess = map(i -> variables[n * (2 * r_f + r_p - 3) + i] - rand(field), 1:Int64(n / 2))

1-element Vector{FqMPolyRingElem}:
 x_3_1 + 5174

In [15]:
gb = groebner_basis_f4(ideal(polynomials) + ideal(guess), nr_thrds=16, info_level=2);


Legend for f4 information
--------------------------------------------------------
deg       current degree of pairs selected in this round
sel       number of pairs selected in this round
pairs     total number of pairs in pair list
mat       matrix dimensions (# rows x # columns)
density   density of the matrix
new data  # new elements for basis in this round
          # zero reductions during linear algebra
time(rd)  time of the current f4 round in seconds given
          for real and cpu time
--------------------------------------------------------

deg     sel   pairs        mat          density            new data         time(rd) in sec (real|cpu)
------------------------------------------------------------------------------------------------------
  3       7       7      15 x 25         15.20%        7 new       0 zero         0.03 | 0.35         
  3       1       1       8 x 13         26.92%        1 new       0 zero         0.01 | 0.16         
  3       1       1       3


--------------- INPUT DATA ---------------
#variables                      13
#equations                      13
#invalid equations               0
field characteristic         10007
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                        16
info level                       2
generate pbm files               0
------------------------------------------

---------------- TIMINGS ----------------
overall(elapsed)        0.13 sec
overall(cpu)            1.47 sec
select                  0.00 sec   0.0%
symbolic prep.          0.00 sec   0.0%
update                  0.00 sec   0.0%
convert                 0.00 sec   0.0%
linear algebra          0.00 sec   0.0%
reduce gb               0.00 sec   0.0%
--------------------------------

In [16]:
for poly in gb
    println(poly)
    println(sep)
end

x_4_1 + 4117*x_4_2 + 253
----------------------------------------------------------------------
x_3_1 + 5174
----------------------------------------------------------------------
x_2_2 + 7710*x_3_2 + 5319
----------------------------------------------------------------------
x_1_2 + 4022*x_2_1 + 7487*x_3_2 + 2701
----------------------------------------------------------------------
x_5_1^2 + 9573*x_4_2*x_5_2 + 3057*x_5_1*x_5_2 + 7183*x_5_2^2 + 4475*x_4_2*x_out_1 + 8298*x_5_1*x_out_1 + 4624*x_5_2*x_out_1 + 2161*x_out_1^2 + 7838*x_4_2 + 6506*x_5_1 + 9402*x_5_2 + 2994*x_out_1 + 7347
----------------------------------------------------------------------
x_4_2*x_5_1 + 6532*x_4_2*x_5_2 + 4149*x_4_2*x_out_1 + 3497*x_4_2 + 4522*x_5_1 + 1784*x_5_2 + 6044*x_out_1 + 4040
----------------------------------------------------------------------
x_4_2^2 + 7100*x_4_2 + 2040*x_5_1 + 5963*x_5_2 + 8045*x_out_1 + 3630
----------------------------------------------------------------------
x_out_1^3 + 4022

Compute the vector space dimension.

In [17]:
A, _ = quo(parent(gb[1]), ideal(gb))
D = factor(vector_space_dimension(A))

1 * 3^8